In [ ]:
import pandas as pd
import re

In [ ]:
df= pd.read_csv('./data/cleaned_hotelreviews.csv')

# Preprocessing

1) Drop all Nan row

In [ ]:
#drop all Nan
df.dropna(inplace= True)

In [ ]:
df

2) Removing stopwords and splitting into corpus

In [ ]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

In [ ]:
df['reviews']= df['reviews'].apply(lambda x: [word for word in x.split() if word not in stop_list])

In [ ]:
df

In [ ]:
type(df['reviews'][0])

3.1) Stemming

In [ ]:
df1=df.copy()

In [ ]:
df1

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
#df_stem= [[stemmer.stem(w) for w in doc] for doc in df['reviews']]

In [ ]:
df1['reviews']=df1['reviews'].apply(lambda doc: [stemmer.stem(w) for w in doc])

In [ ]:
df1

3.2) Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()

In [ ]:
print("rocks :", lemmatizer.lemmatize("rocks"))

In [ ]:
df2=df.copy()

In [ ]:
df2['reviews']=df2['reviews'].apply(lambda doc: [lemmatizer.lemmatize(w) for w in doc])

In [ ]:
df2

# 4) TF-IDF

# UNIGRAM

# 4.1) Create dictionary

In [ ]:
import gensim 
from gensim import corpora

<b> 4.1.1) Create list from dataframe <b>

In [ ]:
#df1 stemming only
reviews= df1['reviews']
doc_stem=reviews.values.tolist()
doc_stem

In [ ]:
#df2 lemmatise only
doc_lem= df2['reviews'].values.tolist()
doc_lem

<b> 4.1.2) Create dictionary <b>

In [ ]:
dict_stem=corpora.Dictionary(doc_stem)

In [ ]:
print(dict_stem)

In [ ]:
dict_lem=corpora.Dictionary(doc_lem)

In [ ]:
print(dict_lem)

In [ ]:
#to see all mappings from tokens to IDs in the dictionary
token_to_id=dict_stem.token2id
print(type(token_to_id))
print(token_to_id)

In [ ]:
token_to_id2=dict_lem.token2id
print(type(token_to_id2))
print(token_to_id2)

<b> 4.2) Represent documents into vectors - TF Models <b>

In [ ]:
vec_stem= [dict_stem.doc2bow(doc) for doc in doc_stem]
vec_stem

In [ ]:
vec_lem= [dict_lem.doc2bow(doc) for doc in doc_lem]

<b> Corpus to TF-IDF vectors <b>

In [ ]:
from gensim import models

In [ ]:
tfidf= models.TfidfModel(vec_stem)
print(tfidf)

In [ ]:
vec_stem_with_tfidf = [tfidf[vec] for vec in vec_stem]
vec_stem_with_tfidf

In [ ]:
# [sg_dictionary.doc2bow(doc) for doc in sg_stem]

In [ ]:
# sg_docs = [sg_corpus.words(f) for f in sg_corpus.fileids()] #Take all files
# sg_lower = [[w.lower() for w in doc] for doc in sg_docs] #Case
# sg_alpha = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in sg_lower] #alphabets
# sg_stop = [[w for w in doc if w not in stop_list] for doc in sg_alpha] #stopwords
# sg_stem = [[stemmer.stem(w) for w in doc] for doc in sg_stop] #stemmng
# sg_dictionary = corpora.Dictionary(sg_stem)
# print(sg_dictionary)

# # Converting some documents in SGNews Apr2012 to vectors and
# # display the first 10 elements of these vectors.
# sg_doc1 = sg_stem[0]
# sg_vec1 = sg_dictionary.doc2bow(sg_doc1)
# print(sg_vec1[0:10])
# sg_doc2 = sg_stem[1]
# sg_vec2= sg_dictionary.doc2bow(sg_doc2)
# print(sg_vec2[0:10])

# # Converting all documents in SGNews Apr2012 to a list of sparse vectors. Store in sg_vecs variable
# sg_vecs = [sg_dictionary.doc2bow(doc) for doc in sg_stem]

In [ ]:
tfidf_lem= models.TfidfModel(vec_lem)
print(tfidf_lem)

In [ ]:
vec_lem_with_tfidf = [tfidf[vec] for vec in vec_lem]

In [ ]:
vec_lem_with_tfidf

# Train the LDA model with Gensim

In [ ]:
import gensim

<b> STEMMING <b>

In [ ]:
lda_reviews_stem= gensim.models.ldamodel.LdaModel(corpus=vec_stem, id2word=dict_stem, num_topics=10)

In [ ]:
topics_stem=lda_reviews_stem.show_topics(10,15)

for i in range(0,10):
    print(topics_stem[i])

<b> LEMMITIZATION <b>

In [ ]:
lda_reviews= gensim.models.ldamodel.LdaModel(corpus=vec_lem, id2word=dict_lem, num_topics=10)

In [ ]:
topics_lem=lda_reviews.show_topics(10,15)

for i in range(0,10):
    print(topics_lem[i])

# Visualisation

In [ ]:
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()

In [ ]:
# # Generate the visuals
# visual= pyLDAvis.gensim.prepare(lda_reviews_stem, vec_stem, dict_stem)

In [ ]:
# pyLDAvis.save_html(visual, "topic_stem.html")

# LDA Mallet Model

<b>STEMMING<b>

In [ ]:
import gensim
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
import os
os.environ.update({'MALLET_HOME':r'/Users/jaslynwong/Softwares/mallet/mallet-2.0.8/'})

no_topics=10

mallet_path = r'/Users/jaslynwong/Softwares/mallet/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=vec_stem, num_topics=no_topics, id2word=dict_stem)


In [ ]:
vecTop= ldamallet.show_topics()
for i in range(0, 10):
    print(vecTop[i])

<b>LEMMITIZATION<b>

In [ ]:
import gensim
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
import os
os.environ.update({'MALLET_HOME':r'/Users/jaslynwong/Softwares/mallet/mallet-2.0.8/'})

# no_topics=10

# mallet_path = r'/Users/jaslynwong/Softwares/mallet/mallet-2.0.8/bin/mallet' # update this path
# ldamallet2 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=vec_lem, num_topics=no_topics, id2word=dict_lem)


In [ ]:
# vecTop= ldamallet2.show_topics()
# for i in range(0, 10):
#     print(vecTop[i])

In [ ]:
ldamallet_lem_tfidf = gensim.models.wrappers.LdaMallet(mallet_path, corpus=vec_lem_with_tfidf, num_topics=no_topics, id2word=dict_lem)



In [ ]:
vecTop= ldamallet_lem_tfidf.show_topics()
for i in range(0, 10):
    print(vecTop[i])

sentence rank, text rank - summarisation 

# Choosing Optimal Number of the Topics

<b>Choose topic for STEMMING<b>

In [ ]:
# Can take a long time to run. In this case we are going to  k_max=10.
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 10, 2):
    #sg_lda_x = gensim.models.ldamodel.LdaModel(corpus=sg_vecs, id2word=sg_dictionary, num_topics=num_topics)
    lda_stem_mallet = gensim.models.wrappers.LdaMallet(mallet_path, iterations=100, corpus=vec_stem, num_topics=no_topics, id2word=dict_stem)
    coherencemodel = CoherenceModel(model=lda_stem_mallet, texts=doc_stem, dictionary=dict_stem, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(lda_stem_mallet)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

In [ ]:
# Show graph

import matplotlib.pyplot as plt

limit=10; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

<b>Choose number of topic for LEMMITIZATION<b>

In [ ]:
# Can take a long time to run. In this case we are going to  k_max=10.
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 12, 2):
    #sg_lda_x = gensim.models.ldamodel.LdaModel(corpus=sg_vecs, id2word=sg_dictionary, num_topics=num_topics)
    lda_lem_mallet = gensim.models.wrappers.LdaMallet(mallet_path, iterations=100, corpus=vec_lem, num_topics=no_topics, id2word=dict_lem)
    coherencemodel = CoherenceModel(model=lda_lem_mallet, texts=doc_lem, dictionary=dict_lem, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(lda_lem_mallet)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

In [ ]:
# Show graph

import matplotlib.pyplot as plt

limit=12; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

<b>MALLET COHERENCE RESULT - choose number of topics for lemmitization<b>

In [ ]:
# Can take a long time to run. In this case we are going to  k_max=10.
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 10, 2):
    #sg_lda_x = gensim.models.ldamodel.LdaModel(corpus=sg_vecs, id2word=sg_dictionary, num_topics=num_topics)
    lda_mallet_lem_tfidf = gensim.models.wrappers.LdaMallet(mallet_path, iterations=100, corpus=vec_lem_with_tfidf, num_topics=no_topics, id2word=dict_lem)
    coherencemodel = CoherenceModel(model=lda_mallet_lem_tfidf, texts=doc_lem, dictionary=dict_lem, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(lda_mallet_lem_tfidf)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

In [ ]:
# Show graph

import matplotlib.pyplot as plt

limit=10; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Coherence Score (Don't use perplexity)

In [ ]:
# # Compute Perplexity for stemming only
# perplex_stem= lda_reviews_stem.log_perplexity(vec_stem, total_docs=None) #For LDAModel
#   # a measure of how good the model is. lower the better.
    
# print('\nPerplexity for LDAModel: ', perplex_stem)

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence score for stemming only
coherence_model_stem = CoherenceModel(model=lda_reviews_stem, texts=doc_stem, dictionary=dict_stem, coherence='c_v')
coherence_lda_stem = coherence_model_stem.get_coherence()
print('\nCoherence Score LDA-stem: ', coherence_lda_stem)

# Compute Coherence score for lemmatising only
coherence_model_lda_lem = CoherenceModel(model=lda_reviews, texts=doc_lem, dictionary=dict_lem, coherence='c_v')
coherence_lda_lem = coherence_model_lda_lem.get_coherence()
print('\nCoherence Score LDA-lem: ', coherence_lda_lem)


In [ ]:
#Compute Coherence score for mallet
from gensim.models import CoherenceModel
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=doc_stem, dictionary=dict_stem, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score LDAMallet: ', coherence_ldamallet)

# Most Dominant Topic for each Document

In [ ]:
#Find most dominant topic

import numpy as np
import pandas as pd

def format_topics_sentences(ldamodel, corpus, data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(data)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

#I choose model_list[1] where the number of topics is 4

#df_topic_sents_keywords = format_topics_sentences(ldamodel=model_list[1], corpus=sg_vecs, data=sg_docs)
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=vec_stem, data=doc_stem)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


In [ ]:
#Find most dominant topic

import numpy as np
import pandas as pd

def format_topics_sentences(ldamodel, corpus, data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(data)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

#I choose model_list[1] where the number of topics is 4

df_topic_sents_keyw = format_topics_sentences(ldamodel=model_list[2], corpus=vec_lem, data=doc_lem)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

# Save model

In [ ]:
#Save a model to disk, or reload a pre-trained model
lda_mallet_stem=ldamallet #Or you can save the best model, model_list[1]

lda_mallet_stem.save("lda_mallet_stem")

lda_disk=gensim.models.wrappers.LdaMallet.load("lda_mallet_stem")